In [4]:
#nbi:hide_in
#@title Default title text
import pandas
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
#!pip install -q lmfit
import lmfit
#!pip install -q numdifftools
import numdifftools
from timeit import default_timer as timer
from lmfit import Parameters, Minimizer, report_fit
import ipywidgets as widgets
from ipywidgets import interact

# New form of data acquisition. It pulls updated data from the github repository used by JHU
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
df_recovered = pandas.read_csv(url, error_bad_lines=False)
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
df_deaths = pandas.read_csv(url, error_bad_lines=False)
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df_confirmed = pandas.read_csv(url, error_bad_lines=False)

#######  Italy data #######
Italy_deaths = df_deaths[(df_deaths['Country/Region'] == 'Italy')][df_deaths.columns.values[4:-1]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == 'Italy')][df_deaths.columns.values[4:-1]].to_numpy(dtype='float') > 0)[1][0]:-1]
Italy_recovered = df_recovered[(df_recovered['Country/Region'] == 'Italy')][df_deaths.columns.values[4:-1]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == 'Italy')][df_deaths.columns.values[4:-1]].to_numpy(dtype='float') > 0)[1][0]:-1]
Italy_confirmed = df_confirmed[(df_confirmed['Country/Region'] == 'Italy')][df_deaths.columns.values[4:-1]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == 'Italy')][df_deaths.columns.values[4:-1]].to_numpy(dtype='float') > 0)[1][0]:-1]
Italy_active = Italy_confirmed - Italy_recovered - Italy_deaths

#######  Mexico data #######
Peru_deaths = df_deaths[(df_deaths['Country/Region'] == 'Peru')][df_deaths.columns.values[4:-1]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == 'Peru')][df_deaths.columns.values[4:-1]].to_numpy(dtype='float') > 0)[1][0]:-1]
Peru_recovered = df_recovered[(df_recovered['Country/Region'] == 'Peru')][df_deaths.columns.values[4:-1]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == 'Peru')][df_deaths.columns.values[4:-1]].to_numpy(dtype='float') > 0)[1][0]:-1]
Peru_confirmed = df_confirmed[(df_confirmed['Country/Region'] == 'Peru')][df_deaths.columns.values[4:-1]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == 'Peru')][df_deaths.columns.values[4:-1]].to_numpy(dtype='float') > 0)[1][0]:-1]
Peru_active = Peru_confirmed - Peru_recovered - Peru_deaths

#data = pandas.read_excel('deaths8.xlsx')
#US_deaths = data['US'].dropna().to_numpy(dtype='float')


def RMPlot(name):
  t = np.linspace(0, int(1.3*len(globals()[str(name)+'_deaths'])), 2000)
  t0 = np.array(range(len(globals()[str(name)+'_deaths'])))
  #print(globals()[str(name)+'_deaths'])

  # fcn2min é uma das funções mais importantes do pacote lmfit. Basicamente ela define o modelo que o minimizador vai utilizar para fitar
  # os pontos dos dados

  def fcn2min(params, t, data):
      """Generalized Richard's Model"""
      K = params['K']
      alpha = params['alpha']
      r = params['r']
      #tc = params['tc']
      tc = np.log((-1+(K/3.1)**alpha)/alpha)/(alpha*r)
      model = K/(1+alpha*np.exp(-alpha*r*(t-tc)))**(1/alpha)
      return model - data


  # A próxima etaa é definir os chutes iniciais. Eu sempre tento fixando os limites ou deixando sem limite


  ##### Parâmetros livres quarantine after day 30

  params = Parameters()
  params.add('K', value=2e4)
  params.add('alpha', value=.1, min=0, max=1)
  params.add('r', value=0.2, min=0)
  #params.add('tc', value=10, min=0)

  ##### Parâmtros limitados quarantine after day 30

  #params = Parameters()
  #params.add('K', value=300000)
  #params.add('alpha', value=.21, min=0., max=1.)
  #params.add('p', value=.999, min=0., max=1.)
  #params.add('r', value=0.5)

  # Criando o minimizador. Eu passo dois métodos de minimização: o Nelder-Mead, lento porém robusto, que dá um chute incial melhor
  # pada o segundo minimizador o LM.

  start =timer()

  # create Minimizer

  minner = Minimizer(fcn2min, params, fcn_args=(t0,globals()[str(name)+'_deaths']))

  # first solve with Nelder-Mead algorithm

  out1 = minner.minimize(method= 'nelder')

  end = timer()
  #print('time spent first minimization:', end - start)

  #lmfit.report_fit(out1)

  start = timer()

  #for p in out1.params:
  #   out1.params[p].stderr = abs(out1.params[p].value * 0.05)

  # then solve with Levenberg-Marquardt using the
  # Nelder-Mead solution as a starting point

  out2 = minner.minimize(method='leastsq', params=out1.params)


  end = timer()
  #print('time spent second minimization:', end - start)

  #lmfit.report_fit(out2)

  params = out2.params
  K = params['K']
  alpha = params['alpha']
  r = params['r']
  #tc = params['tc']

  def func(t,r,alpha,K):
      tc = np.log((-1+(K/3.1)**alpha)/alpha)/(alpha*r)
      #print('C0',globals()[str(name)+'_deaths'][0])
      #print('tc',tc)
      return K/(1+alpha*np.exp(-alpha*r*(t-tc)))**(1/alpha)



  # calculate final result

  # final = data + out2.residual

  ##R^2:
  # print('R2=',1 - out2.residual.var() / np.var(data))

  #### Confidence interval
  #for p in out2.params:
  #   out2.params[p].stderr = abs(out2.params[p].value * 0.05)

  # start = timer()
  #
  # ci, trace = lmfit.conf_interval(minner, out2, sigmas=[1, 2], trace=True)
  # lmfit.printfuncs.report_ci(ci)
  #
  # end = timer()
  # print('time spent confidence interval:', end - start)



  #plt.plot(t0,globals()[str(name)+'_deaths'],'ro', alpha=0.5, label=str(name)+'Deaths')
  #plt.plot(t,func(t,r,alpha,K),'-k', )
  #plt.show()

  fig = plt.figure(facecolor='w')
  ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
  plt.plot(t0,globals()[str(name)+'_deaths'],'ro', alpha=0.5, label=str(name)+'Deaths')
  ax.plot(t,func(t,r,alpha,K), '-k', label='RM \nr= '+str('%.3f' % r)+'\nalpha= '+str('%.3f' % alpha))
  ax.set_xlabel('Days since first death')
  ax.set_ylabel('Deaths')
  ax.set_title(name+' RM fit')
  ax.yaxis.set_tick_params(length=0)
  ax.xaxis.set_tick_params(length=0)
  ax.grid(b=True, which='major', c='w', lw=2, ls='-')
  legend = ax.legend()
  legend.get_frame().set_alpha(0.5)
  for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)
  plt.show()
  return 

interact(RMPlot, name=['Italy', 'Peru']);


interactive(children=(Dropdown(description='name', options=('Italy', 'Peru'), value='Italy'), Output()), _dom_…